### Rolling window calculations - seasonal

Limiting data to just the winter months (May to Oct).  Calculating 2 monthly mean, then find maximum and minimum values, then calculate range.

Export data as a binary for use in Peter Tan's unsupervised learning program.

See the annual version of this code for more commenting

In [ ]:
import builtins
import numpy as np
import xarray as xr
import os
import pandas as pd

#import custom functions
import sys
sys.path.append('src')
import DEAPlotting
from read_envi_header import read_envi_header

In [ ]:
#user inputs
start_year = 2000
end_year = 2018
directory = '/g/data1a/r78/cb3058/dea-notebooks/crop-extents_seasonal/data_dir/griffith/'
index_name = 'brightness'
# 'mndwi' 'brightness' 'msavi'

In [ ]:
z = range(start_year,end_year+1, 1)
years = []
for zz in z:
    years.append(str(zz))

def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

for folder in get_immediate_subdirectories(directory):
    for year in years:
        print("working on year: " + index_name + ", " + year)

        filename_img = directory + year + "/" + index_name + ".img"
        filename_hdr = directory + year + "/" + index_name + ".hdr"
        
        def rollingaverageSeasonal_range(filename_img, filename_hdr, index_name):
            index = xr.open_rasterio(filename_img)
            hdr = read_envi_header(filename_hdr)
            time_str = hdr['band names']
            time = []
            for i in time_str:
                x = pd.to_datetime(i, format='%Y-%m-%d')
                time.append(x)

            index = xr.DataArray(index.values, coords = [time, index.y, index.x],
                                 dims = ['time', 'y', 'x'], name=index_name, attrs=index.attrs)
            
            index = index.where(index>-998, np.nan)
            
            def is_winter(month):
                return (month >= 5) & (month <= 10)

            seasonal_data = index.sel(time=is_winter(index['time.month']))
            
            index_2M = seasonal_data.resample(time='2M').mean('time', skipna=True)

            index_2M_max = index_2M.max('time')
            index_2M_min = index_2M.min('time')
            index_2M_range = index_2M_max - index_2M_min
            index_2M_range = index_2M_range.fillna(-999.) 
            
            index_2M_range.values.tofile(directory + year + "/" + index_name+'_range.img')
        
        rollingaverageSeasonal_range(filename_img, filename_hdr, index_name=index_name)
    break
print('finished')

### visualizing bands

In [ ]:
def rollingaverageSeasonal_range_xr(filename_img, filename_hdr, index_name):
    index = xr.open_rasterio(filename_img)
    hdr = read_envi_header(filename_hdr)
    time_str = hdr['band names']
    time = []
    for i in time_str:
        x = pd.to_datetime(i, format='%Y-%m-%d')
        time.append(x)

    index = xr.DataArray(index.values, coords = [time, index.y, index.x],
                         dims = ['time', 'y', 'x'], name=index_name, attrs=index.attrs)

    index = index.where(index>-998, np.nan)

    def is_winter(month):
        return (month >= 5) & (month <= 10)

    seasonal_data = index.sel(time=is_winter(index['time.month']))
#     print(seasonal_data.time.values)
    index_2M = seasonal_data.resample(time='2M').mean('time', skipna=True)
#     print(index_2M.time.values)
    index_2M_max = index_2M.max('time')
    index_2M_min = index_2M.min('time')
    index_2M_range = index_2M_max - index_2M_min
    index_2M_range = index_2M_range.fillna(-999.) 
    return index_2M_range

In [ ]:
folder = '/g/data1a/r78/cb3058/dea-notebooks/crop-extents_seasonal/data_dir/griffith/2016/'
filename_img = folder + 'brightness.img'
filename_hdr = folder + 'brightness.hdr'

brightness_range = rollingaverageSeasonal_range_xr(filename_img, filename_hdr, index_name='brightness')

In [ ]:
filename_img = folder + 'msavi.img'
filename_hdr = folder + 'msavi.hdr'
msavi_range = rollingaverageSeasonal_range_xr(filename_img, filename_hdr, index_name='msavi')

In [ ]:
filename_img = folder + 'mndwi.img'
filename_hdr = folder + 'mndwi.hdr'
wetness_range = rollingaverageSeasonal_range_xr(filename_img, filename_hdr, index_name='mndwi')

In [ ]:
ds = xr.Dataset({'wetness_range':wetness_range,'msavi_range': msavi_range,'brightness_range': brightness_range})

In [ ]:
DEAPlotting.rgb(ds, bands=['wetness_range', 'msavi_range', 'brightness_range'], size=10)

In [ ]:
def rollingaverage_max_xr(filename_img, filename_hdr, index_name):
    index = xr.open_rasterio(filename_img)
    hdr = read_envi_header(filename_hdr)
    time_str = hdr['band names']
    time = []
    for i in time_str:
        x = pd.to_datetime(i, format='%Y-%m-%d')
        time.append(x)

    index = xr.DataArray(index.values, coords = [time, index.y, index.x],
                         dims = ['time', 'y', 'x'], name=index_name, attrs=index.attrs)

    index = index.where(index>-998, np.nan)

    index_3M = index.resample(time='M').mean('time', skipna=True)
    index_3M_max = index_3M.mean('time')
    index_3M_max = index_3M_max.fillna(-999.) 
    return index_3M_max

In [ ]:
folder = '/g/data1a/r78/cb3058/dea-notebooks/crop-extents/data_dir/griffith/2017/'
filename_img = folder + 'brightness.img'
filename_hdr = folder + 'brightness.hdr'

brightness_max = rollingaverage_max_xr(filename_img, filename_hdr, index_name='brightness')

In [ ]:
brightness_max.plot()

In [ ]:
from __future__ import division
import os
import sys

from osgeo import gdal, gdalconst 
from osgeo.gdalconst import * 


def ENVI_raster_binary_to_2d_array(file_name, parafile):
    
        driver = gdal.GetDriverByName('ENVI')
        driver.Register()

        tmm = pd.read_csv(parafile, header =None)
        rows=tmm.values[1:3][0][0]
        cols=tmm.values[1:3][1][0]
        arr = np.zeros(([nrow,ncol]))
        print(arr.shape)
#         image_array=np.fromfile(filename, dtype=np.float32)
# #         band = inDs.GetRasterBand(1)
#         image_array = [cols, rows]
#         print(image_array.shape)

#         return image_array


# The function can be called as follows:
folder = '/g/data1a/r78/cb3058/dea-notebooks/crop-extents/data_dir/griffith/2017/'
file_name = folder + 'msavi_range.img'
parafile = folder+'/ts_irow_icol.csv'
msavi =  ENVI_raster_binary_to_2d_array(file_name, parafile) 
msavi



In [ ]:
import matplotlib.pyplot as plt
np.fromfile(file_name, dtype=np.float32).shape

In [ ]:
folder = '/g/data1a/r78/cb3058/dea-notebooks/crop-extents/data_dir/griffith/2017/'
file_name = folder + 'msavi_range.img'
xr.open_rasterio(file_name)

 

In [ ]:
#visualizing bands
directory = "/g/data1a/r78/cb3058/dea-notebooks/crop-extents/data_dir/griffith/2017/"
filename= directory + "phg_mndwi.img"
parafile=directory+'/ts_irow_icol.csv'
tmm = pd.read_csv(parafile, header =None)
nrow=tmm.values[1:3][0][0]
ncol=tmm.values[1:3][1][0]

print (nrow, ncol)
pnum = nrow*ncol
data=np.zeros([nrow,ncol])
tgt=4
col=0

def readimgfile(filename, pnum, tgt, data, col):
    """
    Opens a .img envi file and returns a np array
    """
    #print(filename)
    imgdata=np.fromfile(filename, dtype=np.float32)
    print(imgdata, imgdata.shape, tgt*pnum)
    oneblock=imgdata[tgt*pnum:(tgt+1)*pnum]
    print(oneblock)
    print(oneblock.shape)
    data[:, col]=imgdata[tgt*pnum:(tgt+1)*pnum]
    return data

x = readimgfile(filename, pnum, tgt, data, col)

In [ ]:
x.shape

In [ ]:
def ENVI_get_geotransform(filename):
    '''
    retruns the geotransform object from
    and envi .hdr file
    '''
    driver = gdal.GetDriverByName('ENVI') 
    driver.Register()

    inDs = gdal.Open(filename, GA_ReadOnly)

    print('get georeference information')
    geotransform = inDs.GetGeoTransform()
    originX = geotransform[0]
    originY = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = geotransform[5]

    return geotransform

def readimgfile(filename, pnum, tgt, data, col):
    """
    Opens a .img envi file and returns a np array
    """
    #print(filename)
    imgdata=np.fromfile(filename, dtype=np.float32)
    #print(imgdata, imgdata.shape, tgt*pnum)
    oneblock=imgdata[tgt*pnum:(tgt+1)*pnum]
    #print(oneblock)
    #print(oneblock.shape)
    data[:, col]=imgdata[tgt*pnum:(tgt+1)*pnum]
    
directory = "/g/data1a/r78/cb3058/dea-notebooks/crop-extents/data_dir/griffith/2000/"
filename= directory + "phg_msavi.img"
parafile=directory+'/ts_irow_icol.csv'
tmm = pd.read_csv(parafile, header =None)
nrow=tmm.values[1:3][0][0]
ncol=tmm.values[1:3][1][0]
print (nrow, ncol)
pnum = nrow*ncol
tgt=4
col=0

readimgfile(filename, pnum, tgt, data, col)
geotransform =  ENVI_get_geotransform(file_name) 

In [ ]:
#visualizing bands
directory = "/g/data1a/r78/cb3058/dea-notebooks/crop-extents/data_dir/griffith/2017/"
filename= directory + "msavi_range.img"
parafile=directory+'/ts_irow_icol.csv'
tmm = pd.read_csv(parafile, header =None)
nrow=tmm.values[1:3][0][0]
ncol=tmm.values[1:3][1][0]

print (nrow, ncol)
pnum = nrow*ncol
data=np.zeros([pnum, 5])
tgt=4
col=0

def readimgfile(filename, pnum, tgt, data, col):
    """
    Opens a .img envi file and returns a np array
    """
    #print(filename)
    imgdata=np.fromfile(filename, dtype=np.float32)
    #print(imgdata, imgdata.shape, tgt*pnum)
    oneblock=imgdata[tgt*pnum:(tgt+1)*pnum]
    #print(oneblock)
    #print(oneblock.shape)
    data[:, col]=imgdata[tgt*pnum:(tgt+1)*pnum]

readimgfile(filename, pnum, tgt, data, col)



In [ ]:
def three_band_image(ds, bands=['red', 'green', 'blue'], time=0, figsize=(10, 10), title='Time',
                     projection='projected', contrast_enhance=False, reflect_stand=5000):


# Create new numpy array matching shape of xarray
t, y, x = ds[bands[0]].shape
rawimg = np.zeros((y, x, 3), dtype=np.float32)

# Add xarray bands for a given time into three dimensional numpy array
for i, colour in enumerate(bands):

rawimg[:, :, i] = ds[colour][time].values


# Create new numpy array matching shape of xarray
y, x = ds[bands[0]].shape
rawimg = np.zeros((y, x, 3), dtype=np.float32)

# Add xarray bands into three dimensional numpy array
for i, colour in enumerate(bands):

    rawimg[:, :, i] = ds[colour].values
            
# Set nodata value to NaN
rawimg[rawimg == -999] = np.nan

# Optionally compute contrast based on histogram
if contrast_enhance:

    # Stretch contrast using histogram
    img_toshow = exposure.equalize_hist(rawimg, mask=np.isfinite(rawimg))

else:

    # Stretch contrast using defined reflectance standardisation and clip to between 0 and 1
    # to prevent warnings; defaults to reflect_stand = 5000
    img_toshow = (rawimg / reflect_stand).clip(0, 1)

# Plot figure, setting x and y axes from extent of xarray dataset
fig, ax = plt.subplots(figsize=figsize)
plt.imshow(img_toshow)